In [ ]:
# # Connect to Google Drive
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [99]:
# import dependencies
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import string
import re
plt.style.use('ggplot')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [115]:
# read in csv
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/Final-Project/Resource/IMDb_movies.csv")

# df = pd.read_csv("Resource/IMDB_movies.csv", low_memory=False)
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [116]:
# clean the data
# extract only the column we want:
# Movie title/Rating/Duration/Release Date/Genres
# quanitfy string
movies_cols = ["imdb_title_id", "title", "year", "genre", "description", "metascore"]
df_movies = df[movies_cols].copy()

print(f"Shape: {str(df_movies.shape)}")
df_movies.head()

Shape: (85855, 6)


,imdb_title_id,title,year,genre,description,metascore
0,tt0000009,Miss Jerry,1894,Romance,The adventures of a female reporter in the 1890s.,NaN
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",True story of notorious Australian outlaw Ned ...,NaN
2,tt0001892,Den sorte drøm,1911,Drama,Two men of high rank are both wooing the beaut...,NaN
3,tt0002101,Cleopatra,1912,"Drama, History",The fabled queen of Egypt's affair with Roman ...,NaN
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",Loosely adapted from Dante's Divine Comedy and...,NaN


In [117]:
df_movies.dropna(inplace=True)

print(f"Shape: {str(df_movies.shape)}")
df_movies.head()

Shape: (13279, 6)


,imdb_title_id,title,year,genre,description,metascore
76,tt0006864,Intolerance,1916,"Drama, History","The story of a poor young woman, separated by ...",99.0
506,tt0017136,Metropolis,1927,"Drama, Sci-Fi",In a futuristic city sharply divided between t...,98.0
566,tt0018037,Il cantante di jazz,1927,"Drama, Music, Musical",The son of a Jewish Cantor must defy the tradi...,66.0
628,tt0018773,Il circo,1928,"Comedy, Romance",The Tramp finds work and the girl of his dream...,90.0
714,tt0019777,The Cocoanuts,1929,"Comedy, Musical","During the Florida land boom,",69.0


In [111]:
# function for text cleaning 
def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower()
    
    return text

def clean_title(name):
    # remove backslash-apostrophe 
    name = re.sub("\'", "", name) 
    # remove everything except alphabets 
    name = re.sub("[^a-zA-Z]"," ",name) 
    # remove whitespaces 
    name = ' '.join(name.split()) 
    # convert text to lowercase 
    # text = text.lower()
    
    return name


def lower_case(case):
    case = case.lower()

    return case

In [118]:
#  df_movies["description"] = df_movies["description"].apply(lambda x: clean_text(x))
df_movies["title"] = df_movies["title"].apply(lambda x: clean_title(x))
df_movies["year"] = df_movies["year"].astype(str)
df_movies["genre"] =  df_movies["genre"].apply(lambda x: lower_case(x))
df_movies.head()
 

,imdb_title_id,title,year,genre,description,metascore
76,tt0006864,Intolerance,1916,"drama, history","The story of a poor young woman, separated by ...",99.0
506,tt0017136,Metropolis,1927,"drama, sci-fi",In a futuristic city sharply divided between t...,98.0
566,tt0018037,Il cantante di jazz,1927,"drama, music, musical",The son of a Jewish Cantor must defy the tradi...,66.0
628,tt0018773,Il circo,1928,"comedy, romance",The Tramp finds work and the girl of his dream...,90.0
714,tt0019777,The Cocoanuts,1929,"comedy, musical","During the Florida land boom,",69.0


In [119]:
df_movies["words"] = df_movies["title"] + " " + df_movies["description"]
df_movies["words"] = df_movies["words"].apply(lambda x: clean_text(x))

df_movies.head()

,imdb_title_id,title,year,genre,description,metascore,words
76,tt0006864,Intolerance,1916,"drama, history","The story of a poor young woman, separated by ...",99.0,intolerance the story of a poor young woman se...
506,tt0017136,Metropolis,1927,"drama, sci-fi",In a futuristic city sharply divided between t...,98.0,metropolis in a futuristic city sharply divide...
566,tt0018037,Il cantante di jazz,1927,"drama, music, musical",The son of a Jewish Cantor must defy the tradi...,66.0,il cantante di jazz the son of a jewish cantor...
628,tt0018773,Il circo,1928,"comedy, romance",The Tramp finds work and the girl of his dream...,90.0,il circo the tramp finds work and the girl of ...
714,tt0019777,The Cocoanuts,1929,"comedy, musical","During the Florida land boom,",69.0,the cocoanuts during the florida land boom


In [120]:
stop_words = set(stopwords.words("english"))

def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

df_movies["words"] = df_movies["words"].apply(lambda x: remove_stopwords(x))

In [85]:
df_movies.head()

,imdb_title_id,title,year,genre,description,metascore,words
76,tt0006864,intolerance,1916,"drama, history",the story of a poor young woman separated by p...,99.0,intolerance story poor young woman separated p...
506,tt0017136,metropolis,1927,"drama, sci-fi",in a futuristic city sharply divided between t...,98.0,metropolis futuristic city sharply divided wor...
566,tt0018037,il cantante di jazz,1927,"drama, music, musical",the son of a jewish cantor must defy the tradi...,66.0,il cantante di jazz son jewish cantor must def...
628,tt0018773,il circo,1928,"comedy, romance",the tramp finds work and the girl of his dream...,90.0,il circo tramp finds work girl dreams circus
714,tt0019777,the cocoanuts,1929,"comedy, musical",during the florida land boom,69.0,cocoanuts florida land boom


In [121]:
df_movies = df_movies.drop(["metascore", "year", "description"], axis=1)
df_movies.head()

,imdb_title_id,title,genre,words
76,tt0006864,Intolerance,"drama, history",intolerance story poor young woman separated p...
506,tt0017136,Metropolis,"drama, sci-fi",metropolis futuristic city sharply divided wor...
566,tt0018037,Il cantante di jazz,"drama, music, musical",il cantante di jazz son jewish cantor must def...
628,tt0018773,Il circo,"comedy, romance",il circo tramp finds work girl dreams circus
714,tt0019777,The Cocoanuts,"comedy, musical",cocoanuts florida land boom


In [122]:
# movie_id, plot, movie_name, genre_new

df_movies = df_movies.rename(columns = {"imdb_title_id": "movie_id", "title": "movie_name", "words": "plot"})
df_movies = df_movies.reset_index()
df_movies.head()

,index,movie_id,movie_name,genre,plot
0,76,tt0006864,Intolerance,"drama, history",intolerance story poor young woman separated p...
1,506,tt0017136,Metropolis,"drama, sci-fi",metropolis futuristic city sharply divided wor...
2,566,tt0018037,Il cantante di jazz,"drama, music, musical",il cantante di jazz son jewish cantor must def...
3,628,tt0018773,Il circo,"comedy, romance",il circo tramp finds work girl dreams circus
4,714,tt0019777,The Cocoanuts,"comedy, musical",cocoanuts florida land boom


In [123]:
df_movies.to_csv("/content/gdrive/MyDrive/Colab Notebooks/Final-Project/Resource/IMDB_eliot3.csv")